# nf-loto-platform / TSFM統合版 総合検証ノート

このノートブックは、`nf-loto-platform` の TSFM 統合および ML 解析周辺の追加機能が
意図どおりに動作しているかを**手元の環境で再現可能に検証する**ためのものです。

主な内容:
- 開発環境のセットアップ（`pip install -e .[dev]`）
- AutoModel / TSFM レジストリの一覧テーブル出力
- メトリクス (`smape`, `mape`, `mae`, `rmse`, `pinball_loss`, `coverage`) の動作確認
- レポート生成 (`summarize_forecast_df`, `save_eval_report_html/json`) の確認
- `run_loto_experiment` をスタブ環境で実行するエンドツーエンド検証
- pytest を用いた自動テスト実行のテンプレート


## 1. 開発環境セットアップ

ローカル環境で実行する場合は、まず以下のセルを **必要に応じて** 実行して
依存関係をインストールしてください（社内環境のポリシーに合わせて編集してください）。

In [1]:
# 開発環境セットアップ例（必要に応じてコメントアウトを外してください）
# このセルはローカル環境で実行することを想定しています。
# ※ インターネットに接続できない環境では、事前に wheel を用意してください。

# %pip install -U pip
# %pip install -e ".[dev]"


## 2. ライブラリ読み込み & バージョン確認

プロジェクトおよび主要ライブラリのバージョンを確認します。

In [2]:
import sys
import platform

import pandas as pd

import nf_loto_platform  # プロジェクトのトップレベルパッケージ
from nf_loto_platform.ml import model_registry
from nf_loto_platform.ml_analysis import metrics, reporting

print("Python:", sys.version)
print("Platform:", platform.platform())
print("nf_loto_platform:", getattr(nf_loto_platform, "__version__", "(no __version__)"))

Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
Platform: Windows-10-10.0.26200-SP0
nf_loto_platform: (no __version__)


## 3. AutoModel / TSFM レジストリの可視化

In [3]:
from nf_loto_platform.ml.model_registry import AUTO_MODEL_REGISTRY, list_tsfm_models, list_neuralforecast_models
import pandas as pd

records = []
for name, spec in AUTO_MODEL_REGISTRY.items():
    records.append(
        {
            "name": name,
            "family": spec.family,
            "engine_kind": spec.engine_kind,
            "engine_name": spec.engine_name,
            "is_zero_shot": spec.is_zero_shot,
            "requires_api_key": spec.requires_api_key,
            "context_length": spec.context_length,
            "futr_exog": spec.exogenous.futr,
            "hist_exog": spec.exogenous.hist,
            "stat_exog": spec.exogenous.stat,
        }
    )

registry_df = pd.DataFrame.from_records(records).sort_values("name").reset_index(drop=True)
registry_df


,name,family,engine_kind,engine_name,is_zero_shot,requires_api_key,context_length,futr_exog,hist_exog,stat_exog
0,AutoLSTM,RNN,neuralforecast,None,False,False,NaN,True,True,True
1,AutoMLP,MLP,neuralforecast,None,False,False,NaN,True,True,True
2,AutoMLPMultivariate,MLP,neuralforecast,None,False,False,NaN,True,True,True
3,AutoNBEATS,MLP,neuralforecast,None,False,False,NaN,False,False,False
4,AutoNHITS,MLP,neuralforecast,None,False,False,NaN,True,True,True
5,AutoPatchTST,Transformer,neuralforecast,None,False,False,NaN,False,False,False
6,AutoRNN,RNN,neuralforecast,None,False,False,NaN,True,True,True
7,AutoTFT,Transformer,neuralforecast,None,False,False,NaN,True,True,True
8,AutoTimeMixer,MLP,neuralforecast,None,False,False,NaN,False,False,False
9,Chronos2-ZeroShot,TSFM,tsfm,chronos2,True,False,512.0,False,True,False


### 3-1. TSFM モデル / NeuralForecast モデルの一覧

`list_tsfm_models()` と `list_neuralforecast_models()` の戻り値を確認します。

In [4]:
from nf_loto_platform.ml.model_registry import list_tsfm_models, list_neuralforecast_models

print("TSFM models:", list_tsfm_models())
print("NeuralForecast models:", list_neuralforecast_models())


TSFM models: ['Chronos2-ZeroShot', 'TempoPFN-ZeroShot', 'TimeGPT-ZeroShot']
NeuralForecast models: ['AutoLSTM', 'AutoMLP', 'AutoMLPMultivariate', 'AutoNBEATS', 'AutoNHITS', 'AutoPatchTST', 'AutoRNN', 'AutoTFT', 'AutoTimeMixer']


## 4. メトリクス関数の動作確認

In [5]:
import numpy as np
from nf_loto_platform.ml_analysis import metrics

y = np.array([0.0, 1.0, 2.0, 3.0])
yhat = np.array([0.1, 1.1, 1.9, 2.7])

print("smape:", metrics.smape(y, yhat))
print("mape:", metrics.mape(y + 1e-6, yhat + 1e-6))  # 0 除算回避
print("mae:", metrics.mae(y, yhat))
print("rmse:", metrics.rmse(y, yhat))

# pinball loss の q 変化に対する単調性
for q in [0.1, 0.5, 0.9]:
    val = metrics.pinball_loss(y, yhat, q=q)
    print(f"pinball_loss(q={q}):", val)

# coverage
lower = yhat - 0.5
upper = yhat + 0.5
cov = metrics.coverage(y, lower, upper)
print("coverage (±0.5):", cov)


smape: 56.294577591130604
mape: 2500006.2499960423
mae: 0.15
rmse: 0.17320508075688767
pinball_loss(q=0.1): 0.09499999999999999
pinball_loss(q=0.5): 0.075
pinball_loss(q=0.9): 0.05500000000000002
coverage (±0.5): 1.0


## 5. レポート生成ユーティリティの確認

In [6]:
from pathlib import Path
import tempfile
import pandas as pd
from nf_loto_platform.ml_analysis.reporting import summarize_forecast_df, save_eval_report_html, save_eval_report_json

df = pd.DataFrame(
    {
        "y": [1.0, 2.0, 3.0],
        "y_hat": [0.9, 2.1, 3.2],
    }
)
metrics_dict = summarize_forecast_df(df)
print("summarized metrics:", metrics_dict)

tmpdir = Path(tempfile.mkdtemp())
html_path = tmpdir / "eval_report.html"
json_path = tmpdir / "eval_report.json"

save_eval_report_html(metrics_dict, html_path)
save_eval_report_json(metrics_dict, json_path)

print("HTML report:", html_path)
print("JSON report:", json_path)

# JSON を読み戻して DataFrame として確認
json_df = pd.read_json(json_path)
display(json_df)


summarized metrics: {'mae': 0.13333333333333341, 'rmse': 0.14142135623730961, 'smape': 7.285325798494045}
HTML report: C:\Users\HASHIM~1.RYO\AppData\Local\Temp\tmpo7jqbwep\eval_report.html
JSON report: C:\Users\HASHIM~1.RYO\AppData\Local\Temp\tmpo7jqbwep\eval_report.json


,value
mae,0.133333
rmse,0.141421
smape,7.285326


## 6. `run_loto_experiment` のスタブ付きエンドツーエンド検証

In [7]:
import numpy as np
import pandas as pd

from nf_loto_platform.ml import model_runner, automodel_builder
from nf_loto_platform.db import loto_repository

# --- 極小パネルデータを構築 ---
H = 5
unique_ids = ["nb_series_1", "nb_series_2"]
dates = pd.date_range("2024-01-01", periods=20, freq="D")

records = []
for uid in unique_ids:
    values = np.linspace(0.0, 1.0, num=len(dates))
    for ds, y in zip(dates, values):
        records.append({"unique_id": uid, "ds": ds, "y": float(y)})
panel_df = pd.DataFrame.from_records(records)

# --- DB リポジトリのスタブ ---
def _fake_load_panel_by_loto(table_name: str, loto: str, **kwargs):
    assert table_name == "nf_loto_panel"
    unique_ids_arg = kwargs.get("unique_ids")
    assert set(unique_ids_arg) == set(unique_ids)
    return panel_df

loto_repository.load_panel_by_loto = _fake_load_panel_by_loto

# --- AutoModel / NeuralForecast のスタブ ---
class _DummyAutoModel:
    def __init__(self, h: int):
        self.h = h

    def fit(self, dataset, val_size: int | None = None):
        self._fitted = True
        self._last_val_size = val_size

    def predict(self, dataset):
        # horizon 分の未来 ds を生成し、0.0 の y_hat を返すだけ
        last = panel_df["ds"].max()
        future_dates = pd.date_range(last + pd.Timedelta(days=1), periods=self.h, freq="D")
        preds = []
        for uid in unique_ids:
            for ds in future_dates:
                preds.append({"unique_id": uid, "ds": ds, "y_hat": 0.0})
        return pd.DataFrame.from_records(preds)

def _fake_build_auto_model(
    model_name: str,
    backend: str,
    h: int,
    loss_name: str,
    num_samples: int,
    search_space=None,
    early_stop=None,
    early_stop_patience_steps: int = 3,
    verbose: bool = True,
):
    assert h == H
    return _DummyAutoModel(h=h)

def _fake_build_neuralforecast(model, freq: str, local_scaler_type: str | None):
    # 本来は NeuralForecast インスタンスを返すが、ここでは model をそのまま返す
    return model

automodel_builder.build_auto_model = _fake_build_auto_model
automodel_builder.build_neuralforecast = _fake_build_neuralforecast

preds, meta = model_runner.run_loto_experiment(
    table_name="nf_loto_panel",
    loto="loto6",
    unique_ids=unique_ids,
    model_name="AutoNHITS",
    backend="local",
    horizon=H,
    loss="mae",
    metric="mae",
    num_samples=1,
    cpus=1,
    gpus=0,
    search_space=None,
    freq="D",
    local_scaler_type="robust",
    val_size=7,
    refit_with_val=False,
    use_init_models=False,
    early_stop=True,
    early_stop_patience_steps=1,
)

display(preds.head())
print("meta:", meta)


C:\nf\nf_loto_workspace_ws4_tsfm_ai_scientist_augmented_v4\src\nf_loto_platform\db\loto_repository.py:106: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


DatabaseError: Execution failed on sql '
        SELECT *
        FROM nf_loto_panel
        WHERE loto = %s
          AND unique_id IN (%s,%s)
        ORDER BY unique_id, ds
    ': リレーション"nf_loto_panel"は存在しません
LINE 3:         FROM nf_loto_panel
                     ^


## 7. pytest による自動テスト実行テンプレート

以下はローカル環境で `pytest` を実行するためのサンプルです。

In [ ]:
# ローカル環境でのみ実行してください（本ノートブック上ではコメントアウト）
# %pytest -q

# あるいはメトリクスまわりだけ:
# %pytest tests/ml_analysis -q
